[View in Colaboratory](https://colab.research.google.com/github/battlerhythm/tensorflow/blob/master/9.2%20GAN2.ipynb)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28*28
n_noise = 128            # 생성자의 입력값으로 사용할 노이즈 크기입니다.
n_class = 10


X = tf.placeholder(tf.float32, [None, n_input])

# 노이즈와 실제 이미지에 각각에 해당하는 숫자를 힌트로 넣어주는 용도로 사용할 플레이스 홀더 Y
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

# 변수들을 선언하지 않고 tf.layers를 사용해보겠습니다.
# tf.layers를 사용하면 변수를 선언하지 않고, tf.variable_scope를 이용해 스코프를 지정해 줄수 있습니다.
# 이렇게 하면 나중에 이 스코프에 해당하는 변수만 따로 불러올 수 있습니다.
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
    return output

def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        # 출력값에 활성화 함수를 사용하지 않았는데, 앞서와는 다르게 손실값 계산에
        # Sigmoid.cross_entropy_with_logits함수를 사용하기 위함입니다.
        output = tf.layers.dense(hidden, 1, activation=None)
    return output
    
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=(batch_size, n_noise))

G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

# 구분자 Discriminator는 
# 진짜는 진짜라고 알도록 -> D_real goes 1
# 가짜(생성된것)는 가짜라고 알도록 -> D_gene goes 0
# loss_D 값을 최소화 하면 구분자를 학습 시킬수 있습니다.
# ones_like, zeros_like 함수는 1, 0으로 채운 값들과 비교하는 함수입니다.
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene

# 생성자 Generator는
# 가짜(생성된것)를 진짜처럼 만들도록 -> D_gene goes 1 
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [3]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

!mkdir samples2

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Y: batch_ys, Z: noise})
        
    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y: mnist.test.labels[:sample_size], Z: noise})
    
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
    
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
    
print('최적화 완료!')

Epoch: 0000 D loss: 0.004927 G loss: 8.518
Epoch: 0001 D loss: 0.006159 G loss: 7.373
Epoch: 0002 D loss: 0.01664 G loss: 6.797
Epoch: 0003 D loss: 0.0209 G loss: 7.069
Epoch: 0004 D loss: 0.09996 G loss: 7.684
Epoch: 0005 D loss: 0.02283 G loss: 6.346
Epoch: 0006 D loss: 0.09133 G loss: 6.597
Epoch: 0007 D loss: 0.1172 G loss: 7.708
Epoch: 0008 D loss: 0.1394 G loss: 6.6
Epoch: 0009 D loss: 0.1992 G loss: 5.629
Epoch: 0010 D loss: 0.3148 G loss: 3.881
Epoch: 0011 D loss: 0.2919 G loss: 5.371
Epoch: 0012 D loss: 0.2928 G loss: 4.239
Epoch: 0013 D loss: 0.4302 G loss: 3.994
Epoch: 0014 D loss: 0.3234 G loss: 3.766
Epoch: 0015 D loss: 0.5051 G loss: 3.825
Epoch: 0016 D loss: 0.3446 G loss: 3.905
Epoch: 0017 D loss: 0.4928 G loss: 3.516
Epoch: 0018 D loss: 0.543 G loss: 3.704
Epoch: 0019 D loss: 0.6409 G loss: 3.78
Epoch: 0020 D loss: 0.423 G loss: 3.217
Epoch: 0021 D loss: 0.6339 G loss: 3.564
Epoch: 0022 D loss: 0.7405 G loss: 3.105
Epoch: 0023 D loss: 0.5864 G loss: 2.885
Epoch: 0024 D